# Chapter 03: 자동 미분 (Automatic Differentiation)

## 학습 목표
- 경사하강법을 위한 미분의 필요성을 수학적으로 이해한다
- `tf.GradientTape`의 동작 원리와 사용법을 익힌다
- 고차 미분(Hessian)을 구현할 수 있다
- GradientTape으로 Keras 없이 선형 회귀를 수동 구현한다
- `@tf.function`의 역할과 사용 시점을 이해한다

## 목차
1. [수학적 기초: 편미분과 연쇄 법칙](#수학적-기초)
2. [왜 미분이 필요한가?](#왜-미분인가)
3. [tf.GradientTape 기본 사용](#gradienttape-기초)
4. [고차 미분](#고차-미분)
5. [선형 회귀 수동 구현](#선형-회귀)
6. [@tf.function 소개](#tf-function)
7. [요약](#요약)

## 수학적 기초 <a name='수학적-기초'></a>

### 1. 편미분 (Partial Derivative)

다변수 함수 $f(x_1, x_2, \ldots, x_n)$에서 특정 변수 $x_i$에 대한 편미분은  
나머지 변수를 **상수로 고정**했을 때 $x_i$가 변화할 때 $f$의 변화율입니다:

$$\frac{\partial f}{\partial x_i} = \lim_{\Delta x_i \to 0} \frac{f(x_1, \ldots, x_i + \Delta x_i, \ldots, x_n) - f(x_1, \ldots, x_i, \ldots, x_n)}{\Delta x_i}$$

**그래디언트(Gradient):** 모든 변수에 대한 편미분 벡터

$$\nabla f = \left( \frac{\partial f}{\partial x_1}, \frac{\partial f}{\partial x_2}, \ldots, \frac{\partial f}{\partial x_n} \right)$$

---

### 2. 연쇄 법칙 (Chain Rule)

복합 함수 $L = L(y)$, $y = y(w)$가 있을 때, 연쇄 법칙:

$$\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial w}$$

신경망에서는 이 규칙이 층을 거슬러 올라가며 반복적으로 적용됩니다 (역전파):

$$\frac{\partial L}{\partial w_1} = \frac{\partial L}{\partial a_n} \cdot \frac{\partial a_n}{\partial a_{n-1}} \cdots \frac{\partial a_2}{\partial a_1} \cdot \frac{\partial a_1}{\partial w_1}$$

---

### 3. 경사하강법 (Gradient Descent)

파라미터 $w$를 손실 $L$이 감소하는 방향으로 반복 업데이트:

$$w_{t+1} = w_t - \eta \cdot \frac{\partial L}{\partial w_t}$$

여기서 $\eta$는 학습률(learning rate)입니다.

---
### 🐣 초등학생을 위한 미분·경사하강법 친절 설명!

#### 📐 미분(Derivative)이 뭐예요?

미분은 **'지금 이 순간 얼마나 빠르게 변하고 있는지'** 알려줘요!

> 💡 **비유**: 자동차 속도계처럼요!
> - 속도계 = 거리의 미분 (거리가 시간에 따라 얼마나 빨리 변하는지)
> - 딥러닝에서 미분 = 손실이 파라미터에 따라 어떻게 변하는지

#### 🏔️ 경사하강법이 뭐예요?

손실 함수(Loss)를 **산(mountain)**이라고 생각해보세요!
우리의 목표는 **가장 낮은 골짜기**(최솟값)를 찾는 것이에요.

```
    손실
     |
     |   ↘ 내리막길 (기울기)
     |        ↘
     |              ★ 최솟값 목표!
     └───────────────────── 파라미터(w)
```

**경사하강법의 원리:**
1. 현재 위치에서 경사(기울기, Gradient)를 측정
2. 경사의 반대 방향으로 한 걸음 이동
3. 1~2 반복 → 점점 낮은 곳으로!

$$w_{새} = w_{현재} - \eta \times \text{기울기}$$

- $\eta$ (에타): **학습률** — 한 걸음의 크기 (너무 크면 왔다갔다, 너무 작으면 너무 느림!)

#### 🎬 GradientTape — 연산 '녹화' 후 '재생'

GradientTape은 이름 그대로 **'기울기 측정 테이프'**예요!

```python
with tf.GradientTape() as tape:  # 🎬 녹화 시작
    y = f(x)                     # 연산 기록됨
grad = tape.gradient(y, x)       # ⏪ 역방향으로 재생 → 기울기 계산!
```

> 💡 마치 블랙박스처럼 전방향 주행을 기록하고,
> 역방향으로 돌려서 '어디서 무슨 일이 있었는지' 분석하는 거예요!

In [ ]:
import tensorflow as tf
import numpy as np
import matplotlib
matplotlib.use('Agg')  # 헤드리스 환경을 위한 백엔드 설정
import matplotlib.pyplot as plt

print(f"TensorFlow 버전: {tf.__version__}")

## 왜 미분이 필요한가? <a name='왜-미분인가'></a>

딥러닝 학습의 핵심은 **손실 함수(Loss Function)를 최소화**하는 것입니다.  
이를 위해 파라미터(가중치)를 어떤 방향으로 얼마나 움직여야 하는지 알아야 합니다.  
그 방향을 알려주는 것이 바로 **그래디언트(기울기)**입니다.

In [ ]:
# 직관적 이해: 1D 함수에서 최솟값 찾기
# f(x) = x^2 - 4x + 5 → 최솟값: x=2 에서 f(2)=1

def loss_fn(x):
    return x**2 - 4*x + 5

# x 범위에서 함수값 계산
x_vals = tf.linspace(-1.0, 5.0, 100)
y_vals = loss_fn(x_vals)

print("f(x) = x² - 4x + 5")
print(f"f(-1) = {loss_fn(tf.constant(-1.0)).numpy():.2f}")
print(f"f( 0) = {loss_fn(tf.constant(0.0)).numpy():.2f}")
print(f"f( 2) = {loss_fn(tf.constant(2.0)).numpy():.2f}  ← 최솟값 (f'(2)=0)")
print(f"f( 4) = {loss_fn(tf.constant(4.0)).numpy():.2f}")

# 미분: f'(x) = 2x - 4 → f'(2) = 0 (최솟값 조건)
print("\n미분 f'(x) = 2x - 4")
for x in [-1.0, 0.0, 1.0, 2.0, 3.0, 4.0]:
    grad = 2*x - 4
    direction = "← 왼쪽으로 이동 필요" if grad > 0 else ("→ 오른쪽으로 이동 필요" if grad < 0 else "★ 최솟값!")
    print(f"  f'({x:.0f}) = {grad:+.1f}  {direction}")

## tf.GradientTape 기본 사용 <a name='gradienttape-기초'></a>

`tf.GradientTape`은 **자동 미분(Automatic Differentiation)** 엔진입니다.  
테이프(Tape) 비유: 연산을 테이프에 녹화하듯 기록한 뒤, 역방향으로 재생하며 그래디언트를 계산합니다.

```python
with tf.GradientTape() as tape:
    y = f(x)   # 연산 기록
dy_dx = tape.gradient(y, x)   # 역방향으로 그래디언트 계산
```

In [ ]:
# 기본 사용: 스칼라 함수의 그래디언트
# f(x) = x^2, f'(x) = 2x

x = tf.Variable(3.0)   # Variable이어야 자동으로 추적됨

with tf.GradientTape() as tape:
    # GradientTape 컨텍스트 내의 Variable 연산이 기록됨
    y = x ** 2

# 테이프를 역방향으로 재생하여 dy/dx 계산
dy_dx = tape.gradient(y, x)

print(f"f(x) = x²")
print(f"x = {x.numpy()}")
print(f"f(x) = {y.numpy()}")
print(f"f'(x) = dy/dx = {dy_dx.numpy()}  (수학적 정답: 2×3 = 6)")

In [ ]:
# 다변수 함수의 편미분
# f(x, y) = x^2 + 2xy + y^3
# ∂f/∂x = 2x + 2y
# ∂f/∂y = 2x + 3y^2

x = tf.Variable(2.0)
y = tf.Variable(3.0)

with tf.GradientTape() as tape:
    f = x**2 + 2*x*y + y**3

# 두 변수에 대한 편미분을 한 번에 계산
grads = tape.gradient(f, [x, y])
df_dx, df_dy = grads

print(f"f(x, y) = x² + 2xy + y³")
print(f"x={x.numpy()}, y={y.numpy()}")
print(f"f({x.numpy()}, {y.numpy()}) = {f.numpy():.1f}")
print(f"\n편미분 결과:")
print(f"  ∂f/∂x = 2x + 2y  = 2×{x.numpy()}+2×{y.numpy()} = {2*x+2*y:.1f}")
print(f"  GradientTape 결과: {df_dx.numpy():.1f}")
print(f"\n  ∂f/∂y = 2x + 3y² = 2×{x.numpy()}+3×{y.numpy()}² = {2*x+3*y**2:.1f}")
print(f"  GradientTape 결과: {df_dy.numpy():.1f}")

In [ ]:
# tf.constant는 기본적으로 추적되지 않음 → tape.watch()로 명시적 추적

x_const = tf.constant(2.0)   # constant

with tf.GradientTape() as tape:
    tape.watch(x_const)   # constant를 명시적으로 추적
    y = x_const ** 3      # f(x) = x^3, f'(x) = 3x^2

dy_dx = tape.gradient(y, x_const)
print(f"f(x) = x³, x = {x_const.numpy()}")
print(f"f'(x) = 3x² = 3×{x_const.numpy()}² = {3*x_const.numpy()**2:.1f}")
print(f"GradientTape 결과: {dy_dx.numpy():.1f}")

# persistent=True: 여러 번 gradient() 호출 허용
x = tf.Variable(2.0)

with tf.GradientTape(persistent=True) as tape:
    y1 = x ** 2      # f1(x) = x²
    y2 = x ** 3      # f2(x) = x³

grad_y1 = tape.gradient(y1, x)   # 2x = 4
grad_y2 = tape.gradient(y2, x)   # 3x² = 12
del tape   # persistent 테이프는 사용 후 명시적으로 삭제

print(f"\npersistent=True 예시:")
print(f"  d(x²)/dx at x=2: {grad_y1.numpy()}  (예상: 4)")
print(f"  d(x³)/dx at x=2: {grad_y2.numpy()}  (예상: 12)")

## 고차 미분 <a name='고차-미분'></a>

GradientTape을 중첩하면 **2차 미분(Hessian)** 등 고차 미분을 계산할 수 있습니다.

$$f(x) = x^4 \Rightarrow f'(x) = 4x^3 \Rightarrow f''(x) = 12x^2$$

In [ ]:
# 2차 미분 (이중 GradientTape)
# f(x) = x^4
# f'(x) = 4x³  → x=2: 4×8 = 32
# f''(x) = 12x² → x=2: 12×4 = 48

x = tf.Variable(2.0)

with tf.GradientTape() as tape2:      # 외부 테이프: 2차 미분용
    with tf.GradientTape() as tape1:  # 내부 테이프: 1차 미분용
        y = x ** 4
    dy_dx = tape1.gradient(y, x)      # 1차 미분: 4x³

d2y_dx2 = tape2.gradient(dy_dx, x)   # 2차 미분: 12x²

print(f"f(x) = x⁴, x = {x.numpy()}")
print(f"f(x)   = {y.numpy():.1f}        (= 2⁴ = 16)")
print(f"f'(x)  = {dy_dx.numpy():.1f}       (= 4×2³ = 32)")
print(f"f''(x) = {d2y_dx2.numpy():.1f}       (= 12×2² = 48)")

In [ ]:
# Hessian 행렬 계산
# f(x, y) = x^2 + xy + y^2
# Hessian H = [[∂²f/∂x², ∂²f/∂x∂y], [∂²f/∂y∂x, ∂²f/∂y²]]
#           = [[2, 1], [1, 2]]

x = tf.Variable(1.0)
y = tf.Variable(1.0)

with tf.GradientTape() as t2:
    with tf.GradientTape() as t1:
        f = x**2 + x*y + y**2
    # 1차 편미분
    grads = t1.gradient(f, [x, y])   # [∂f/∂x, ∂f/∂y]

# 2차 편미분 (Hessian 대각 원소)
# 주의: 비대각 원소는 별도의 persistent tape 필요
grad_x, grad_y = grads
print(f"f(x,y) = x² + xy + y² at x=1, y=1")
print(f"f(1,1) = {f.numpy():.1f}")
print(f"\n1차 편미분:")
print(f"  ∂f/∂x = 2x + y = {grad_x.numpy():.1f}  (예상: 3)")
print(f"  ∂f/∂y = x + 2y = {grad_y.numpy():.1f}  (예상: 3)")

# 이론적 Hessian
print(f"\n이론적 Hessian 행렬:")
print(f"  H = [[∂²f/∂x², ∂²f/∂x∂y], [∂²f/∂y∂x, ∂²f/∂y²]]")
print(f"    = [[2, 1], [1, 2]]")

## 선형 회귀 수동 구현 <a name='선형-회귀'></a>

이제 GradientTape을 이용해 Keras 없이 선형 회귀를 처음부터 구현합니다.

**모델:** $\hat{y} = wx + b$

**손실 함수 (MSE):** $L = \frac{1}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)^2 = \frac{1}{n}\sum_{i=1}^{n}(y_i - wx_i - b)^2$

**그래디언트:**
$$\frac{\partial L}{\partial w} = -\frac{2}{n}\sum_{i=1}^{n} x_i(y_i - \hat{y}_i)$$

$$\frac{\partial L}{\partial b} = -\frac{2}{n}\sum_{i=1}^{n}(y_i - \hat{y}_i)$$

**파라미터 업데이트 (Gradient Descent):**
$$w \leftarrow w - \eta \frac{\partial L}{\partial w}, \quad b \leftarrow b - \eta \frac{\partial L}{\partial b}$$

In [ ]:
# 1단계: 학습 데이터 생성
# 실제 관계: y = 2x + 1 + 노이즈

tf.random.set_seed(42)
np.random.seed(42)

N = 100  # 데이터 포인트 수

# 입력 x: 균등분포에서 샘플링
X = tf.cast(tf.random.uniform([N, 1], -3, 3), tf.float32)

# 목표값 y: y = 2x + 1 + 가우시안 노이즈
TRUE_W = 2.0
TRUE_B = 1.0
noise = tf.random.normal([N, 1], stddev=0.5)
Y = TRUE_W * X + TRUE_B + noise

print(f"학습 데이터 생성 완료")
print(f"  X shape: {X.shape}, 범위: [{X.numpy().min():.2f}, {X.numpy().max():.2f}]")
print(f"  Y shape: {Y.shape}, 범위: [{Y.numpy().min():.2f}, {Y.numpy().max():.2f}]")
print(f"  실제 파라미터: w={TRUE_W}, b={TRUE_B}")
print(f"  학습 목표: 이 값에 가깝게 수렴하는지 확인")

In [ ]:
# 2단계: 모델 파라미터 초기화
# tf.Variable로 선언해야 GradientTape이 추적 가능

w = tf.Variable(tf.random.normal([1, 1]), name='weight')   # 무작위 초기화
b = tf.Variable(tf.zeros([1]), name='bias')                # 0으로 초기화

print(f"파라미터 초기화:")
print(f"  w 초기값: {w.numpy().flatten()[0]:.4f}  (목표: {TRUE_W})")
print(f"  b 초기값: {b.numpy()[0]:.4f}  (목표: {TRUE_B})")

In [ ]:
# 3단계: 학습 루프 구현
# GradientTape으로 그래디언트 계산 → 파라미터 업데이트 반복

learning_rate = 0.05
num_epochs = 200

loss_history = []
w_history = []
b_history = []

for epoch in range(num_epochs):
    
    # ── 순전파 & 그래디언트 계산 ──────────────────────────
    with tf.GradientTape() as tape:
        # 순전파: y_hat = w*x + b
        y_pred = tf.matmul(X, w) + b
        
        # 손실 계산: MSE = (1/n) * sum((y - y_hat)^2)
        loss = tf.reduce_mean(tf.square(Y - y_pred))
    
    # ── 역전파: 그래디언트 계산 ───────────────────────────
    # dL/dw, dL/db 자동 계산 (연쇄 법칙 자동 적용)
    gradients = tape.gradient(loss, [w, b])
    dL_dw, dL_db = gradients
    
    # ── 파라미터 업데이트 (경사하강법) ────────────────────
    # w ← w - η * dL/dw
    # b ← b - η * dL/db
    w.assign_sub(learning_rate * dL_dw)
    b.assign_sub(learning_rate * dL_db)
    
    # 기록
    loss_history.append(loss.numpy())
    w_history.append(w.numpy().flatten()[0])
    b_history.append(b.numpy()[0])
    
    # 진행 상황 출력 (매 50 에포크)
    if (epoch + 1) % 50 == 0:
        print(f"Epoch {epoch+1:3d}/{num_epochs}  | "
              f"Loss: {loss.numpy():.6f} | "
              f"w: {w.numpy().flatten()[0]:.4f} | "
              f"b: {b.numpy()[0]:.4f}")

print(f"\n학습 완료!")
print(f"  최종 w: {w.numpy().flatten()[0]:.4f}  (실제: {TRUE_W})")
print(f"  최종 b: {b.numpy()[0]:.4f}  (실제: {TRUE_B})")
print(f"  최종 손실: {loss_history[-1]:.6f}")

In [ ]:
# 학습 결과 시각화
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

# 1. 손실 곡선
axes[0].plot(loss_history, 'b-', linewidth=1.5)
axes[0].set_xlabel('에포크', fontsize=11)
axes[0].set_ylabel('MSE 손실', fontsize=11)
axes[0].set_title('학습 손실 곡선', fontsize=12)
axes[0].grid(True, alpha=0.3)
axes[0].set_yscale('log')  # 로그 스케일로 수렴 확인

# 2. 가중치 w 수렴
axes[1].plot(w_history, 'r-', linewidth=1.5, label='학습된 w')
axes[1].axhline(y=TRUE_W, color='g', linestyle='--', label=f'실제 w={TRUE_W}')
axes[1].set_xlabel('에포크', fontsize=11)
axes[1].set_ylabel('w 값', fontsize=11)
axes[1].set_title('가중치 w 수렴', fontsize=12)
axes[1].legend(fontsize=9)
axes[1].grid(True, alpha=0.3)

# 3. 데이터와 학습된 직선
x_line = np.linspace(-3, 3, 100)
y_line = w.numpy().flatten()[0] * x_line + b.numpy()[0]
y_true_line = TRUE_W * x_line + TRUE_B

axes[2].scatter(X.numpy(), Y.numpy(), alpha=0.4, s=15, label='데이터', color='steelblue')
axes[2].plot(x_line, y_line, 'r-', linewidth=2, label=f'학습 결과: y={w.numpy().flatten()[0]:.2f}x+{b.numpy()[0]:.2f}')
axes[2].plot(x_line, y_true_line, 'g--', linewidth=1.5, label=f'실제: y={TRUE_W}x+{TRUE_B}')
axes[2].set_xlabel('x', fontsize=11)
axes[2].set_ylabel('y', fontsize=11)
axes[2].set_title('선형 회귀 결과', fontsize=12)
axes[2].legend(fontsize=8)
axes[2].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('/Users/alex/AI_TestPlayground/chapter01_basics/linear_regression_result.png', dpi=100, bbox_inches='tight')
plt.close()
print("그래프 저장됨: chapter01_basics/linear_regression_result.png")

In [ ]:
# 비교: Keras optimizer 사용 버전
# 위의 수동 업데이트를 Keras optimizer로 단순화

# 파라미터 재초기화
tf.random.set_seed(42)
w2 = tf.Variable(tf.random.normal([1, 1]))
b2 = tf.Variable(tf.zeros([1]))

# Adam optimizer 사용 (현대 딥러닝에서 가장 많이 사용)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.1)

for epoch in range(200):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X, w2) + b2
        loss2 = tf.reduce_mean(tf.square(Y - y_pred))
    
    grads2 = tape.gradient(loss2, [w2, b2])
    
    # optimizer.apply_gradients: 수동 assign_sub 대신 optimizer가 처리
    optimizer.apply_gradients(zip(grads2, [w2, b2]))

print(f"Keras Adam optimizer 결과:")
print(f"  w: {w2.numpy().flatten()[0]:.4f}  (실제: {TRUE_W})")
print(f"  b: {b2.numpy()[0]:.4f}  (실제: {TRUE_B})")
print(f"  최종 손실: {loss2.numpy():.6f}")
print(f"\n수동 구현과 결과 비교:")
print(f"  수동 w: {w.numpy().flatten()[0]:.4f} vs Adam w: {w2.numpy().flatten()[0]:.4f}")

## @tf.function 소개 <a name='tf-function'></a>

`@tf.function`은 Python 함수를 **TensorFlow 계산 그래프로 컴파일**하는 데코레이터입니다.

- **Eager Execution:** Python 코드가 줄 단위로 즉시 실행 → 디버깅 편리
- **@tf.function:** 함수 전체를 그래프로 컴파일 → 실행 속도 최적화

**트레이싱(Tracing):** 첫 호출 시 함수를 그래프로 변환, 이후 호출에서 그래프 재사용

In [ ]:
# @tf.function 기본 사용

# Eager 버전
def train_step_eager(X, Y, w, b, lr=0.01):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X, w) + b
        loss = tf.reduce_mean(tf.square(Y - y_pred))
    grads = tape.gradient(loss, [w, b])
    w.assign_sub(lr * grads[0])
    b.assign_sub(lr * grads[1])
    return loss

# @tf.function 버전 — 동일한 코드에 데코레이터만 추가
@tf.function
def train_step_graph(X, Y, w, b, lr=0.01):
    with tf.GradientTape() as tape:
        y_pred = tf.matmul(X, w) + b
        loss = tf.reduce_mean(tf.square(Y - y_pred))
    grads = tape.gradient(loss, [w, b])
    w.assign_sub(lr * grads[0])
    b.assign_sub(lr * grads[1])
    return loss

# 성능 비교
import time

tf.random.set_seed(42)
w_e = tf.Variable(tf.random.normal([1, 1]))
b_e = tf.Variable(tf.zeros([1]))
w_g = tf.Variable(w_e.numpy())  # 동일한 초기값으로 시작
b_g = tf.Variable(b_e.numpy())

# 워밍업 (JIT 컴파일)
_ = train_step_eager(X, Y, w_e, b_e)
_ = train_step_graph(X, Y, w_g, b_g)

N = 500
t0 = time.time()
for _ in range(N):
    train_step_eager(X, Y, w_e, b_e)
t_eager = time.time() - t0

t0 = time.time()
for _ in range(N):
    train_step_graph(X, Y, w_g, b_g)
t_graph = time.time() - t0

print(f"학습 스텝 {N}회 성능 비교:")
print(f"  Eager (일반 함수):    {t_eager:.4f}초")
print(f"  @tf.function (그래프): {t_graph:.4f}초")
speedup = t_eager / t_graph
print(f"  속도 비율: {speedup:.2f}x {'(그래프가 빠름)' if speedup > 1 else '(유사)'}")

In [ ]:
# @tf.function 주의사항: Python 사이드 이펙트

# 트레이싱(tracing) 횟수 확인
trace_count = 0

@tf.function
def traced_fn(x):
    global trace_count
    trace_count += 1  # 트레이싱 시에만 실행됨 (그래프 실행 시에는 무시)
    print(f"Python 코드 실행 (트레이싱): trace_count={trace_count}")
    return x * 2

# 동일 타입/shape: 재사용
print("첫 번째 호출 (float32):")
r1 = traced_fn(tf.constant(1.0))
print("두 번째 호출 (동일 타입 — 재추적 없음):")
r2 = traced_fn(tf.constant(2.0))
print("세 번째 호출 (int32 — 새로운 타입이므로 재추적):")
r3 = traced_fn(tf.constant(3))

print(f"\n결론: trace_count={trace_count} (타입이 바뀔 때만 재추적)")
print("tf.print를 사용하면 그래프 실행 중에도 출력 가능:")

@tf.function
def fn_with_tfprint(x):
    tf.print("그래프 실행 중 x =", x)  # 그래프 실행 시에도 실행됨
    return x ** 2

fn_with_tfprint(tf.constant(5.0))

## 요약 <a name='요약'></a>

### 핵심 수식 정리

| 개념 | 수식 | 의미 |
|------|------|------|
| 편미분 | $\frac{\partial f}{\partial x}$ | 다른 변수 고정 후 $x$ 변화에 대한 $f$의 변화율 |
| 그래디언트 | $\nabla f = (\frac{\partial f}{\partial x_1}, \ldots, \frac{\partial f}{\partial x_n})$ | 함수의 최대 증가 방향 |
| 연쇄 법칙 | $\frac{\partial L}{\partial w} = \frac{\partial L}{\partial y} \cdot \frac{\partial y}{\partial w}$ | 역전파의 수학적 근거 |
| 경사하강법 | $w \leftarrow w - \eta \nabla_w L$ | 손실 감소 방향으로 파라미터 업데이트 |
| MSE 손실 | $L = \frac{1}{n}\sum_i(y_i - \hat{y}_i)^2$ | 예측과 실제 값의 평균 제곱 오차 |

### GradientTape 핵심 패턴

```python
# 기본 패턴
with tf.GradientTape() as tape:
    loss = forward_pass(x, w)
grads = tape.gradient(loss, w)
w.assign_sub(learning_rate * grads)
```

### 다음 챕터: 실습 퀴즈

`practice/ex01_tensor_quiz.ipynb`에서 다음을 복습합니다:
- 텐서 shape 계산 문제
- 브로드캐스팅 결과 예측
- transpose 결과 계산
- GradientTape으로 임의 함수 미분
- `tf.Variable` vs `tf.constant` 실습